## Bubble Map of Applicant Start-Ups

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

%matplotlib inline

#read in data that was cleaned in 'XRC-data cleaning' notebook
df = pd.read_csv('dataname')
df.rename(columns={'Startup or Person': 'TeamSolo'}, inplace=True)
df.rename(columns={'Average Score': 'InitScore'}, inplace=True)

## Tally Applications by Location

In [3]:
from collections import Counter

#Set all nans to be empty so all cells can be 'counted' in the 'Tags' column
df['Location'].copy()[df['Location'].isnull()] = ""

locales = Counter()
for f in df['Location']:
    locales[f] +=1

counterdata = pd.DataFrame.from_dict(locales, orient='index').reset_index()
counterdata = counterdata.rename(columns={'index':'Location', 0:'count'})

## New data frame with count for each city

In [4]:
uniquecities = df.drop_duplicates(['Location'])
mapdata = pd.merge(uniquecities, counterdata, on ='Location')
mapdata = mapdata[['Lat', 'Long', 'Location', 'count']]
mapdata.to_csv('data/mapdata.csv')

uniquecountries = df.drop_duplicates(['Country'])
len(uniquecountries)

76

## Initializing plotly offline

In [5]:
import plotly.plotly as py
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [6]:
mapdata['text'] = mapdata['Location'] + ' ' + (mapdata['count'].astype(str)) + '<br> Companies'
limits = [(0,2),(3,10),(11,20),(21,50),(51,210)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","rgb(0,128,0)"]
cities = []
scale = 5000 #not relevant for this map

for i in range(len(limits)):
    lim = limits[i]
    mapdata_sub = mapdata[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'world',
        lon = mapdata_sub['Long'],
        lat = mapdata_sub['Lat'],
        text = mapdata_sub['text'],
        marker = dict(
            size = mapdata_sub['count'],
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)
    
    layout = dict(
        title = 'Applicants for [Startup Accelerator]: 2016-2017<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='world',
            projection=dict( type='world' ),
            showland = True,
            landcolor = 'rgb(211,211,211)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )
fig = dict(data=cities, layout=layout)
iplot(fig, validate=False)